<a class="anchor" id="0.1"></a>
# **Introduction**

Tracking number is a set of number which help customers to track their packages' location after giving it to a courier company.
In order to confirm what packages are shipped, staffs of a courier company need to take pictures of the bill and record the tracking numbers.
To ease up the checking, this programe is created to capture the tacking number in the pictures and compare to the record of the courier data to confirm packages shipped.

##**Objective**
Reocrd tracking number from the pdf image and convert to an excel file

##**Input**
pdf files with image content

##**Output**
Excel file with list of tracking numbers

##**Steps**
- read pdf image
- convert to plain text
- extract the tracking number
- append on the list
- export to an excel

In [1]:
# Link to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Import library
## keep silent
%%capture
!pip install PyMuPDF
import fitz  # PyMuPDF

!apt install tesseract-ocr # installs the Tesseract OCR engine
!pip install pytesseract # installs the Python wrapper
import pytesseract

from PIL import Image
import os
import re
import pandas as pd

In [9]:
# Define folder
pdf_folder='/content/gdrive/My Drive/pdf_image/'

# Initialize empty list
track_list = []

In [10]:
# Create pdf list
all_pdf = os.listdir(pdf_folder)

# Filter for PDF files
pdf_files = [file for file in all_pdf if file.endswith(".pdf")]
pdf_index = 1

# Define regex pattern
regex = r'XX\s*\d+'

# Process each PDF file
for pdf_file in pdf_files:
    pdf = fitz.open(os.path.join(pdf_folder,  pdf_file)) # open a document
    track_list.append(f'doc{pdf_index}')

    for pages in range(len(pdf)): # iterate over pdf pages
        page = pdf[pages] # get the page
        image_list = page.get_images()

        for img in image_list:
            xref = img[0] # get the XREF of the image
            pix = fitz.Pixmap(pdf, xref) # create a Pixmap

            if pix.n - pix.alpha > 3: # CMYK: convert to RGB first
               pix = fitz.Pixmap(fitz.csRGB, pix)

            text = pytesseract.image_to_string(Image.frombytes("RGB", (pix.width, pix.height), pix.samples)) # Use pytesseract to do OCR on the image
            pix = None # reset pix

            track_no = re.findall(regex, text)[0].replace(" ", "")
            track_list.append(track_no)

    pdf_index += 1

In [11]:
# Define excel file path
excel_path = os.path.join(pdf_folder + 'track_no.xlsx')

df = pd.DataFrame(track_list)

# Export to excel file
df.to_excel(excel_path, index=False, header=False)